In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import cvxpy
import math
import numpy as np
#import cubic_spline_planner


NX = 4  # x = x, y, v, yaw
NU = 2  # a = [accel, steer]
T = 5  # horizon length

# mpc parameters
R = np.diag([0.01, 0.01])  # input cost matrix
Rd = np.diag([0.01, 1.0])  # input difference cost matrix
Q = np.diag([1.0, 1.0, 0.5, 0.5])  # state cost matrix
Qf = Q  # state final matrix

GOAL_DIS = 1.5  # goal distance
STOP_SPEED = 0.5 / 3.6  # stop speed
MAX_TIME = 500.0  # max simulation time

# iterative paramter
MAX_ITER = 3  # Max iteration
DU_TH = 0.1  # iteration finish param

TARGET_SPEED = 15#10.0 / 3.6  # [m/s] target speed
N_IND_SEARCH = 10  # Search index number

DT = 0.2  # [s] time tick

# Vehicle parameters
WB = 110#2.5  # [m]

MAX_STEER = np.deg2rad(45.0)  # maximum steering angle [rad]
MAX_DSTEER = np.deg2rad(30.0)  # maximum steering speed [rad/s]
MAX_SPEED = 25#55.0 / 3.6  # maximum speed [m/s]
MIN_SPEED = -25#-20.0 / 3.6  # minimum speed [m/s]
MAX_ACCEL = 5.0  # maximum accel [m/ss]

show_animation = True

In [2]:
def iterative_linear_mpc_control(xref, Motion_Status, dref, oa, od):
    """
    MPC contorl with updating operational point iteraitvely
    """  
    
    if oa is None or od is None:
        oa = [0.0] * T
        od = [0.0] * T
    state=Motion_Status.get_state()   
    for i in range(MAX_ITER):
        xbar = Motion_Status.get_trajectory(oa, od,T)
        poa, pod = oa[:], od[:]
        oa, od, ox, oy, oyaw, ov = linear_mpc_control(xref, xbar, state, dref)
        du = sum(abs(oa - poa)) + sum(abs(od - pod))  # calc u change value
        if du <= DU_TH:
            break
    else:
        print("Iterative is max iter")
    
    return oa, od, ox, oy, oyaw, ov,xbar

def get_linear_model_matrix(v, phi, delta):

    A = np.zeros((NX, NX))
    A[0, 0] = 1.0
    A[1, 1] = 1.0
    A[2, 2] = 1.0
    A[3, 3] = 1.0
    A[0, 2] = DT * math.cos(phi)
    A[0, 3] = - DT * v * math.sin(phi)#
    A[1, 2] = DT * math.sin(phi)
    A[1, 3] = DT * v * math.cos(phi)#
    A[3, 2] = DT * math.tan(delta) / WB

    B = np.zeros((NX, NU))
    B[2, 0] = DT
    B[3, 1] = DT * v / (WB * math.cos(delta) ** 2)#

    C = np.zeros(NX)
    C[0] = DT * v * math.sin(phi) * phi
    C[1] = - DT * v * math.cos(phi) * phi
    C[3] = - DT * v * delta / (WB * math.cos(delta) ** 2)
    #C = np.zeros(NX)
    return A, B, C

def linear_mpc_control(xref, xbar, state, dref):
    """
    linear mpc control

    xref: reference point
    xbar: operational point
    x0: initial state
    dref: reference steer angle
    """

    x = cvxpy.Variable((NX, T + 1))
    u = cvxpy.Variable((NU, T))

    cost = 0.0
    constraints = []

    for t in range(T):
        cost += cvxpy.quad_form(u[:, t], R)

        if t != 0:
            cost += cvxpy.quad_form(xref[:, t] - x[:, t], Q)

        A, B, C = get_linear_model_matrix(xbar[2, t], xbar[3, t], 0)
        constraints += [x[:, t + 1] == A * x[:, t] + B * u[:, t]+ C]

        if t < (T - 1):
            cost += cvxpy.quad_form(u[:, t + 1] - u[:, t], Rd)
            constraints += [cvxpy.abs(u[1, t + 1] - u[1, t]) <=
                            MAX_DSTEER * DT]

    cost += cvxpy.quad_form(xref[:, T] - x[:, T], Qf)

    constraints += [x[:, 0] == [state.x,state.y,state.v,state.yaw]]
    constraints += [x[2, :] <= MAX_SPEED]
    constraints += [x[2, :] >= MIN_SPEED]
    constraints += [cvxpy.abs(u[0, :]) <= MAX_ACCEL]
    constraints += [cvxpy.abs(u[1, :]) <= MAX_STEER]

    prob = cvxpy.Problem(cvxpy.Minimize(cost), constraints)
    prob.solve(solver=cvxpy.OSQP, warm_start=True)
    #prob.solve(solver=cvxpy.ECOS, verbose=False)

    if prob.status == cvxpy.OPTIMAL or prob.status == cvxpy.OPTIMAL_INACCURATE:
        ox = np.array(x.value[0, :]).flatten()
        oy = np.array(x.value[1, :]).flatten()
        ov = np.array(x.value[2, :]).flatten()
        oyaw = np.array(x.value[3, :]).flatten()
        oa = np.array(u.value[0, :]).flatten()
        odelta = np.array(u.value[1, :]).flatten()

    else:
        print("Error: Cannot solve mpc..")
        oa, odelta, ox, oy, oyaw, ov = None, None, None, None, None, None

    return oa, odelta, ox, oy, oyaw, ov



In [3]:
class State:
     
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.predelta = None

class Motion():

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.state=State(x=x, y=y, yaw=yaw, v=v)
        
    def get_state(self):
        return State(x=self.state.x, y=self.state.y, yaw=self.state.yaw, v=self.state.v)
        
    def set_state(self,state):
        self.state=state
    def get_next_state(self, a, delta, curr_state=-1):
        if curr_state==-1:
            curr_state=self.state
        # input check
        if delta >= MAX_STEER:
            delta = MAX_STEER
        elif delta <= -MAX_STEER:
            delta = -MAX_STEER

        next_x = curr_state.x + curr_state.v * math.cos(curr_state.yaw) * DT
        next_y = curr_state.y + curr_state.v * math.sin(curr_state.yaw) * DT
        next_yaw = curr_state.yaw + curr_state.v / WB * math.tan(delta) * DT
        next_v = curr_state.v + a * DT
        
        if next_v > MAX_SPEED:
            next_v = MAX_SPEED
        elif next_v < MIN_SPEED:
            next_v = MIN_SPEED

        return State(x=next_x,y=next_y,v=next_v,yaw=next_yaw)


    def get_trajectory(self,accels, deltas,time_step):
        xbar =np.zeros((NX, time_step + 1))
        xbar[:, 0] = self.state.x,self.state.y,self.state.v,self.state.yaw
        #_state=State(x=state.x,y=state.y,v=state.v,yaw=state.yaw)
        _state=Motion.get_state(self)
        for (accel, delta, i) in zip(accels, deltas, range(1, T + 1)):
            next_state = Motion.get_next_state(self,accel, delta, curr_state=_state)
            xbar[:, i] = next_state.x,next_state.y,next_state.v,next_state.yaw
            _state=next_state
        

        return xbar


In [4]:
from Car_Env import Car_Env
from Data_Processor import Data_Processor
from Car_Plot import Move_Contour,Draw_Car

In [5]:
#cx, cy, cyaw, ck = get_switch_back_course(dl)
dl=1.0
rec=True
max_len=350
qlen=25
Env=Car_Env(rec=rec,max_len=max_len,qlen=qlen)
[init_state,map_points]=Env.reset()

ix,iy,iyaw,ivel=init_state[0]
_traj=init_state[1][0]
_dind=init_state[1][1]
_end=init_state[1][2]
ofx,ofy=init_state[2]
img=init_state[3]
Motion_Status = Motion(x=ix, y=iy, yaw=iyaw, v=ivel)
state=Motion_Status.get_state()
#print(init_state)

Track generation: 1255..1573 -> 318-tiles track


In [6]:
if rec==True:
    import os
    if not os.path.exists('Rec/'):
        os.mkdir('Rec')
    if not os.path.exists('Fig/'):
        os.mkdir('Fig')


In [7]:
import matplotlib.gridspec as gridspec
time = 0.0
x = [state.x]
y = [state.y]
yaw = [state.yaw]
v = [state.v]
t = [0.0]
d = [0.0]
a = [0.0]

log_state=[]
log_traj=[]
log_img=[]
log_act=[]
log_next_state=[]


#index, _ = get_closest_point(state, _cx, _cy, cyaw, 0)
odelta, oa = None, None
_index=0
#cyaw = smooth_yaw(cyaw)
accl,gas,steer=0,0,0
state_str=''
act_str=''
xref_str=''
xbar_str=''


_Data_Processor=Data_Processor(_speed=TARGET_SPEED,tlen=qlen)
acc_st=State(x=0,y=0)
itr=0
#fig,axes=plt.subplots(nrows=2,ncols=1,figsize=(10,10))
fig=plt.figure(figsize=(10,10))
gs = fig.add_gridspec(3, 1)
ax1=fig.add_subplot(gs[0,:])
ax2=fig.add_subplot(gs[1:,:])
axes=[ax1,ax2]

#gridspec.GridSpec(3,2)
#axes[0].subplot2grid((3,2),(0,0))
#axes[1].subplot2grid((3,2),(1,0),colspan=2,rowspan=2)
while MAX_TIME >= time:
        state=Motion_Status.get_state()
        _full_state = [state.x, state.y, state.v, state.yaw]
    
        
        _full_state_traj=_Data_Processor.update_trajectory(_traj)
        xref, dref = _Data_Processor.get_ref_trajectory(state,dl,DT,T)
        
        if _end==True:
            break

        _index+=_dind
        
        oa, odelta, ox, oy, oyaw, ov ,xbar= iterative_linear_mpc_control(xref, Motion_Status, dref, oa, odelta)
        
        state_str=str(state.x)+','+str(state.y)+','+str(state.v)+','+str(np.rad2deg(state.yaw))+'\n'
        xref_str=str(xref[0][0])+','+str(xref[1][0])+','+str(xref[2][0])+','+str(np.rad2deg(xref[3][0]))+','+str(_index)+'\n'
        xbar_str=str(xbar[0][0])+','+str(xbar[1][0])+','+str(xbar[2][0])+','+str(np.rad2deg(xbar[3][0]))+'\n'
        act_str=str(accl)+','+str(gas)+','+str(np.rad2deg(steer))+'\n'
    
        #xref_str=str(xref[3][0])+'\n'
        #xbar_str=str(xbar[3][:])+'\n'
        if odelta is not None:
            di, ai = odelta[0], oa[0]
        accl=ai
        steer=di
        
        #act_str=str(steer)+'\n'
        nst,path,offset,img,rw,ter,info=Env.step([steer,accl,0])
        _traj,_dind,_end=path
        ofx,ofy=offset
        
        gas=Env.gas
        
        next_state=State(x=nst[0],y=nst[1],yaw=nst[2],v=nst[3])
        Motion_Status.set_state(next_state)
        
        log_state.append(_full_state)
        log_traj.append(_full_state_traj)
        log_img.append(img)
        log_act.append([accl,steer,0])
        log_next_state.append(next_state)
        
        time = time + DT

        x.append(ofx)
        y.append(ofy)
        yaw.append(state.yaw)
        v.append(state.v)
        t.append(time)
        d.append(di)
        a.append(ai)

        
        cx=np.array(map_points[:max_len,0])#-ofx
        cy=np.array(map_points[:max_len,1])#-ofy
        
        if show_animation:  # pragma: no cover
            axes[0].cla()
            axes[1].cla()
            axes[0].text(.01,.6,'state : '+state_str,fontsize=10,color='r')
            axes[0].text(.01,.4,'xref : '+xref_str,fontsize=10,color='b')
            axes[0].text(.01,.2,'xbar : '+xbar_str,fontsize=10,color='c')
            axes[0].text(.01,0,'act : '+act_str,fontsize=10,color='g')
            
            if ox is not None:
                axes[1].plot(ofx+ox, ofy+oy, "xr", label="MPC",alpha=1.0)
            axes[1].plot(cx, cy, "-b", label="course")
            axes[1].plot(x, y, "oc", label="trajectory")
            axes[1].plot(xref[0, :]+ofx, xref[1, :]+ofy, "xk", label="xref",alpha=1.0)
            axes[1].plot(cx[_index], cy[_index], "xg", label="target",alpha=1.0)
            #plot_car(state.x, state.y, state.yaw, steer=di)
            
            car_points=Draw_Car(ofx,ofy)
            ang_yaw=-(state.yaw*180.0)/math.pi
            rp=Move_Contour(car_points,0,0,ang_yaw)
            r_car_points=rp.apply()
            r_points=np.array(r_car_points,dtype=np.float64)
            axes[1].plot(r_points[:,0].flatten(),r_points[:,1].flatten(),"-r")
            
            plt.axis("equal")
            plt.grid(True)
            #plt.title("Time[s]:" + str(round(time, 2))
            #          + ", speed[km/h]:" + str(round(state.v * 3.6, 2))
            #          + '\nyaw'+str(state.yaw)
            #          + '\nxref_yaw'+str(xref[3][0])
            #          + '\nsteer'+str(steer))
            #fig.canvas.draw()
            plt.pause(0.0001)
            if itr%25==0:
                plt.savefig('Fig/'+str(int(itr/25))+'.jpg')
            itr+=1
if show_animation:  # pragma: no cover
            plt.close("all")
            plt.subplots()
            plt.plot(cx, cy, "-r", label="spline")
            plt.plot(x, y, "-g", label="tracking")
            plt.grid(True)
            plt.axis("equal")
            plt.xlabel("x[m]")
            plt.ylabel("y[m]")
            plt.legend()

            plt.subplots()
            plt.plot(t, v, "-r", label="speed")
            plt.grid(True)
            plt.xlabel("Time [s]")
            plt.ylabel("Speed [kmh]")

            plt.show()


Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative is max iter
Iterative 

ResetNeeded: Trying to step environment which is currently done. While the monitor is active for CarRacing-v0, you cannot step beyond the end of an episode. Call 'env.reset()' to start the next episode.

In [ ]:
log_dict={
'state':log_state,
'traj':log_traj,
'image':log_img,
'action':log_act,
'next_sate':log_next_state
}

In [3]:
#import pickle
#f=open('dataset','wb')
#pickle.dump(log_dict,f)
#f.close()
itr

NameError: name 'itr' is not defined

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=1.0)
i_yaw=cyaw[0]
cyaw=[yaw-i_yaw for yaw in cyaw]
cyaw

In [17]:
import numpy as np
#log_traj[100]
a=np.arctan2(0,-1)
#np.rad2deg(a)
a

3.141592653589793

In [10]:
np.arctan2(1,1)

0.7853981633974483

In [ ]:
#%matplotlib inline
plt.plot(cx,cy)
plt.show()

In [ ]:
plt.plot(range(len(cyaw)),cyaw)
plt.show()

In [ ]:
dcx=np.gradient(cx)
dcy=np.gradient(cy)
cyaw=[np.arctan2(y,x) for x,y in zip(dcx,dcy)]

In [ ]:
cx, cy, cyaw, ck, s = cubic_spline_planner.calc_spline_course(tx,ty,ds=dl)
yaw0=np.deg2rad(90)
cyaw=[yaw-yaw0 for yaw in cyaw]

In [ ]:
import numpy as np
np.rad2deg(2.64)

In [ ]:
cyaw

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 1
n = 1
np.random.seed(1)
A = 10#np.random.randn(m, n)
b = 5#np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A*x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

In [ ]:
print(x.value)

In [ ]:
x=np.array(range(0,100))
x=x/100

In [ ]:
y=10*x-5
y=np.square(y)
plt.plot(x,y)
plt.show()

In [ ]:
import gym
gym.make('CarRacing-v0')